In [1]:
import whisperx
import torch
import os

# Çalşma dizinini tangible klasörüne ayarla
os.chdir('..')

# Şu anki çalışma dizinini kontrol et
print(os.getcwd())


c:\Users\user\Documents\Python Projects\tangible\venv_whisperx\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\Documents\Python Projects\tangible\venv_whisperx\Lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

In [ ]:
def transcribe_audio(audio_path, output_path, language='tr'):
    # Whisper modelini yükleyin
    model = whisperx.load_model("large")  # "base", "small", "medium", "large" modellerinden birini seçebilirsiniz

    # Transkripsiyon (manuel dil belirtimi)
    result = model.transcribe(audio_path, language=language)

    # Dil modelini yükleyin (transkript ile hizalamak için)
    model_a, metadata = whisperx.load_align_model(language_code=language, device="cuda" if torch.cuda.is_available() else "cpu")

    # Transkripti hizalayın
    result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_path, device="cuda" if torch.cuda.is_available() else "cpu")

    # Konuşmacı ayrıştırması için diarization
    diarization = whisperx.diarize(audio_path)
    result_aligned["speaker_labels"] = diarization.labels

    # Sonuçları kaydet
    with open(output_path, "w", encoding="utf-8") as f:
        for segment in result_aligned["segments"]:
            f.write(f"{segment['start']} - {segment['end']} | {segment['speaker']} | {segment['text']}\n")

    print(f"Transkripsiyon ve konuşmacı ayrıştırması tamamlandı: {output_path}")

# Ses dosyalarınızın bulunduğu klasörün yolunu belirtin
audio_folder = "data/"  # Klasör yolunu güncelleyin

# Sonuçları kaydetmek için bir klasör oluşturun
output_folder = "sonuclar/"
os.makedirs(output_folder, exist_ok=True)

# Klasördeki tüm ses dosyalarını işleyin
for filename in os.listdir(audio_folder):
    if filename.endswith((".mp3", ".wav", ".m4a", ".flac")):
        audio_path = os.path.join(audio_folder, filename)
        output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
        print(f"İşleniyor: {filename}")
        transcribe_audio(audio_path, output_path)
